# Using `await` to create agents

In [1]:
import guidance
guidance.llm = guidance.llms.OpenAI("text-curie-001")

bot_agent = guidance(r"""You are an AI bot:
{{#each 'conversation'}}
User: "{{set 'this.user_text' (await 'user_text')}}"
AI: "{{generate 'this.bot_text'}}"
{{/each}}
""")

user_agent = guidance(r"""You are a human user:
User: "{{conversation[0]["user_text"]}}"
{{#each 'conversation'}}
AI: "{{set 'this.bot_text' (await 'bot_text')}}"
User: "{{generate 'this.user_text'}}"
{{/each}}
""")

user_agent = user_agent(conversation=[{"user_text": "Hello, how are you?"}])
for i in range(10):
    bot_agent = bot_agent(user_text=user_agent["conversation"][-2]["user_text"])
    user_agent = user_agent(bot_text=bot_agent["conversation"][-2]["bot_text"])

# dump the whole conversation
user_agent["conversation"]

[{'user_text': 'Hello, how are you?'},
 {'bot_text': "I'm good, thank you for asking. How about you?",
  'user_text': "I'm good, too. How are you today?"},
 {'bot_text': "I'm good, thank you for asking. I'm curious, do you have any questions for me?",
  'user_text': 'No, not really. How are you?'},
 {'bot_text': "I'm good, thank you for asking. I'm curious, do you have any questions for me?",
  'user_text': 'No, not really. How are you?'},
 {'bot_text': "I'm good, thank you for asking. I'm curious, do you have any questions for me?",
  'user_text': 'No, not really. How are you?'},
 {'bot_text': "I'm good, thank you for asking. I'm curious, do you have any questions for me?",
  'user_text': 'No, not really. How are you?'},
 {'bot_text': "I'm good, thank you for asking. I'm curious, do you have any questions for me?",
  'user_text': 'No, not really. How are you?'},
 {'bot_text': "I'm good, thank you for asking. I'm curious, do you have any questions for me?",
  'user_text': 'No, not real